In [122]:
import requests
from pprint import pprint
import numpy as np
import random
import math
from typing import List, Optional, Tuple
import msgspec

In [152]:
coords = [[19.0, 72.8], [28.6, 77.2], [12.9, 77.5], [13.0, 80.2]]
# coords = [
#     [18.9220, 72.8347],
#     [28.6129, 77.2295],
#     [12.9698, 77.7500],
#     [13.0832, 80.2755],
# ]

# coords = [
#     (40.7128, -74.0060),  # New York City, USA
#     (34.0522, -118.2437),  # Los Angeles, USA
#     (41.8781, -87.6298),  # Chicago, USA
#     (45.5017, -73.5673),  # Montreal, Canada
# ]

OSRM_BASE_URL = "http://router.project-osrm.org"
MATRIX_URL = f"{OSRM_BASE_URL}/table/v1/driving/"
ROUTES_URL = f"{OSRM_BASE_URL}/route/v1/driving/"

# [19.0, 72.8] - Mumbai, Maharashtra, India.
# [28.6, 77.2] - Delhi, India.
# [12.9, 77.5] - Bangalore (Bengaluru), Karnataka, India.
# [13.0, 80.2] - Chennai, Tamil Nadu, India.

In [120]:
# schemas


class Waypoint(msgspec.Struct):
    location: List[float]


class RouteResponse(msgspec.Struct):
    code: str
    waypoints: List[Waypoint]


class DistanceMatrixResponse(msgspec.Struct):
    distances: Optional[List[List[float]]]

In [83]:
def is_valid_coordinate(coordinate):
    # Ensure the coordinate is a list and has exactly two elements
    if not isinstance(coordinate, list) or len(coordinate) != 2:
        return False

    lat, lon = coordinate

    # Ensure latitude and longitude are floats and within valid ranges
    if not isinstance(lat, (int, float)) or not isinstance(lon, (int, float)):
        return False
    if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
        return False

    return True

In [84]:
try:
    for i, coord in enumerate(coords):
        if not is_valid_coordinate(coord):
            raise Exception("Invalid coordinate found", coordinate)
except Exception as error:
    print(error)

print("all well")

all well


In [85]:
def fetch_distance_matrix(
    coordinates: List[List[float]],
) -> Optional[List[List[float]]]:
    params = ""
    for coordinate in coordinates:
        lat, long = coordinate
        params += f"{long},{lat};"

    params = params.rstrip(";")
    url = f"{MATRIX_URL}{params}?annotations=distance&skip_waypoints=true"
    print(url)

    try:
        response = requests.get(url)
        response.raise_for_status()
        parsed_response = msgspec.json.decode(
            response.content, type=DistanceMatrixResponse
        )
        return parsed_response.distances
    except requests.exceptions.RequestException as e:
        print(f"Error : {e}")
        return None

In [87]:
def nearest_neighbor_tsp(distance_matrix):
    n = len(distance_matrix)
    unvisited = set(range(1, n))  # Exclude the starting point (0)
    route = [0]  # Start from the first point
    total_distance = 0

    while unvisited:
        last = route[-1]
        next_point = min(unvisited, key=lambda x: distance_matrix[last][x])
        route.append(next_point)
        unvisited.remove(next_point)
        total_distance += distance_matrix[last][next_point]

    # Return to the starting point
    route.append(0)
    total_distance += distance_matrix[route[-2]][0]

    return route, total_distance

In [88]:
def simulated_annealing_tsp(
    distance_matrix, initial_temp=1000, cooling_rate=0.995, iterations=10000
):
    def calculate_total_distance(route):
        return sum(distance_matrix[route[i - 1]][route[i]] for i in range(len(route)))

    def swap_cities(route):
        i, j = random.sample(range(1, len(route) - 1), 2)
        new_route = route.copy()
        new_route[i], new_route[j] = new_route[j], new_route[i]
        return new_route

    current_route, _ = nearest_neighbor_tsp(distance_matrix)
    current_distance = calculate_total_distance(current_route)
    best_route = current_route
    best_distance = current_distance
    temperature = initial_temp

    for _ in range(iterations):
        new_route = swap_cities(current_route)
        new_distance = calculate_total_distance(new_route)

        if new_distance < current_distance or random.random() < math.exp(
            (current_distance - new_distance) / temperature
        ):
            current_route = new_route
            current_distance = new_distance

            if current_distance < best_distance:
                best_route = current_route
                best_distance = current_distance

        temperature *= cooling_rate

    return best_route, best_distance

In [89]:
def two_opt_tsp(distance_matrix, initial_route=None, max_iterations=1000):
    if initial_route is None:
        initial_route, _ = nearest_neighbor_tsp(distance_matrix)

    def calculate_total_distance(route):
        return sum(distance_matrix[route[i - 1]][route[i]] for i in range(len(route)))

    best_route = initial_route
    best_distance = calculate_total_distance(best_route)
    improved = True
    iterations = 0

    while improved and iterations < max_iterations:
        improved = False
        for i in range(1, len(best_route) - 2):
            for j in range(i + 1, len(best_route) - 1):
                new_route = (
                    best_route[:i] + best_route[i : j + 1][::-1] + best_route[j + 1 :]
                )
                new_distance = calculate_total_distance(new_route)
                if new_distance < best_distance:
                    best_route = new_route
                    best_distance = new_distance
                    improved = True
        iterations += 1

    return best_route, best_distance

In [123]:
def haversine_distance(
    coord1: Tuple[float, float], coord2: Tuple[float, float]
) -> float:
    """Calculate the great circle distance between two points on the earth."""
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of earth in kilometers
    r = 6371

    return r * c

In [135]:
def match_coordinates(
    original_coords: List[Tuple[float, float]],
    snapped_coords: List[Tuple[float, float]],
) -> List[int]:
    """Match snapped coordinates to original coordinates based on closest distance."""
    matched_indices = []
    for snapped_coord in snapped_coords:
        distances = [
            haversine_distance(snapped_coord, orig_coord)
            for orig_coord in original_coords
        ]
        closest_index = distances.index(min(distances))
        matched_indices.append(closest_index)
    return matched_indices


def fetch_routes(coordinates: list):
    params = ";".join(f"{long},{lat}" for lat, long in coordinates)
    url = f"{ROUTES_URL}{params}"
    print(f"Request URL: {url}")

    try:
        response = requests.get(url)
        response.raise_for_status()
        route_data = msgspec.json.decode(response.content, type=RouteResponse)

        print("Parsed route data:", route_data)

        # Extract the original coordinates
        original_coords = [(coord[0], coord[1]) for coord in coordinates]
        print("Original coordinates:")
        pprint(original_coords)

        # Extract the snapped coordinates from the response
        snapped_coords = [
            (waypoint.location[1], waypoint.location[0])
            for waypoint in route_data.waypoints
        ]
        print("Snapped coordinates:")
        pprint(snapped_coords)

        # Match snapped coordinates to original coordinates
        waypoint_order = match_coordinates(original_coords, snapped_coords)

        return waypoint_order
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [136]:
pprint(fetch_routes(coords))

Request URL: http://router.project-osrm.org/route/v1/driving/72.8347,18.922;77.2295,28.6129;77.75,12.9698;80.2755,13.0832
Parsed route data: RouteResponse(code='Ok', waypoints=[Waypoint(location=[72.83374, 18.922257]), Waypoint(location=[77.227663, 28.612962]), Waypoint(location=[77.750001, 12.969822]), Waypoint(location=[80.276113, 13.083277])])
Original coordinates:
[(18.922, 72.8347), (28.6129, 77.2295), (12.9698, 77.75), (13.0832, 80.2755)]
Snapped coordinates:
[(18.922257, 72.83374),
 (28.612962, 77.227663),
 (12.969822, 77.750001),
 (13.083277, 80.276113)]
[0, 1, 2, 3]


In [153]:
matrix = fetch_distance_matrix(coordinates=coords)
if not matrix:
    print("error")
else:
    pprint(matrix)

http://router.project-osrm.org/table/v1/driving/72.8,19.0;77.2,28.6;77.5,12.9;80.2,13.0?annotations=distance&skip_waypoints=true
[[0.0, 1388047.9, 995846.7, 1246260.8],
 [1400477.8, 0.0, 2083839.0, 2121598.0],
 [1005837.6, 2125396.8, 0.0, 343205.8],
 [1251204.0, 2120450.8, 338355.5, 0.0]]


In [154]:
from prettytable import PrettyTable

# Find the shortest route using different TSP algorithms
nearest_route, nearest_distance = nearest_neighbor_tsp(matrix)
simulated_route, simulated_distance = simulated_annealing_tsp(matrix)
optimized_route, optimized_distance = two_opt_tsp(matrix)

# Create a PrettyTable instance
table = PrettyTable()
table.field_names = ["Algorithm", "Route", "Total Distance"]

# Add rows to the table
table.add_row(["Nearest Neighbor", nearest_route, nearest_distance])
table.add_row(["Simulated Annealing", simulated_route, simulated_distance])
table.add_row(["Two-Opt", optimized_route, optimized_distance])

# Print the table
print(table)


+---------------------+-----------------+----------------+
|      Algorithm      |      Route      | Total Distance |
+---------------------+-----------------+----------------+
|   Nearest Neighbor  | [0, 2, 3, 1, 0] |   4859981.1    |
| Simulated Annealing | [0, 1, 3, 2, 0] |   4853839.0    |
|       Two-Opt       | [0, 1, 3, 2, 0] |   4853839.0    |
+---------------------+-----------------+----------------+


**TODO**

1. Understand 3 algos used
2. Implement Lin-Kernighan-Helsgaun (LKH) algo
3. Becnchmark 4 algos. for performamce and accuracy
4. More advanced algos

5. MST for TSP
6. Wahtch yt for same
